# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df= pd.read_csv('IdealCities.csv')
df.head()


,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,2074865,Carnarvon,AU,-24.87,113.63,73.40,100.0,0.0,8.05,1588606465
1,5847411,Kahului,US,20.89,-156.47,75.20,69.0,90.0,14.99,1588606225
2,2071860,Esperance,AU,-33.87,121.90,66.20,77.0,90.0,5.82,1588606467
3,3423146,Ilulissat,GL,69.22,-51.10,24.80,68.0,75.0,16.11,1588606469
4,4030556,Rikitea,PF,-23.12,-134.97,73.94,65.0,100.0,11.97,1588606464


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Store locations and humidities
locations = df[['Lat', 'Lng']]
humidity = df.Humidity

In [5]:
# Create heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=True)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
mask1 = (df['Max Temp'] >= 77) & (df['Max Temp'] < 85)
mask2 = (df['Wind Speed'] > 5) & (df['Wind Speed'] < 10)
mask3 = df.Cloudiness < 20
mask4 = df.Humidity < 70
mask5 = mask1 & mask2 & mask3 & mask4

ideal_cities = df.loc[mask5].reset_index(drop=True)
len(ideal_cities)

6

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#Find a hotel. Change row for lat and lng if no result found
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

lat = ideal_cities["Lat"][1]
lng = ideal_cities["Lng"][1]

params["location"] = f"{lat},{lng}"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

name_address = requests.get(base_url, params=params)
name_address = name_address.json()

name_address['results'][0]['name']

'Hotel Cristo Rei'

In [11]:
#iterrate through all locations
names = []
addresses = []
ratings = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in ideal_cities.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # create url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    
    try:
        names.append(name_address["results"][0]["name"])
        addresses.append(name_address["results"][0]["vicinity"])
        ratings.append(name_address["results"][0]["rating"])
    except (KeyError, IndexError):
        print("No hotels listed nearby.")
        names.append("No listed hotels")
        addresses.append("")
        ratings.append("")

No hotels listed nearby.
No hotels listed nearby.


In [12]:
ideal_cities["Hotel"] = names
ideal_cities["Address"] = addresses
ideal_cities["Rating"] = ratings
ideal_cities.head(15)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel,Address,Rating
0,161943,Gumdag,TM,39.21,54.59,83.05,25.0,0.0,6.04,1588606628,No listed hotels,,
1,3451478,Riachao Das Neves,BR,-11.75,-44.91,84.60,49.0,19.0,5.46,1588606649,Hotel Cristo Rei,"-680, Av. Juarez de Souza, 618, Riachão das Neves",3.6
2,3455020,Parauna,BR,-16.95,-50.45,78.64,46.0,6.0,5.32,1588606738,Hotel Cristo Redentor,"Av. Juscelino Kubitscheck, 2205-2307, Paraúna",4.2
3,3970558,Rio Muerto,MX,27.42,-110.23,78.80,34.0,5.0,9.17,1588606742,No listed hotels,,
4,1256025,Sirur,IN,18.83,74.38,80.85,31.0,0.0,8.95,1588614878,"MARVEL INN, HOTEL & BANQUET HALL","Shree Prestige, Nagar - Pune Rd, Shirur",4.4
5,3347019,Namibe,AO,-15.20,12.15,77.18,66.0,0.0,6.62,1588614935,Hotel ibis Styles Iu Namibe,"Avenida Eduardo Mondlane, Namibe",3.6


In [15]:
# Add marker layer ontop of heat map
#Plot locations with marker popups

#define locations
coordinates = ideal_cities[['Lat', 'Lng']]

#store weather and city info
hotelinfo= []
for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>City</dt><dd>{row.City}</dd>
                <dt>Country</dt><dd>{row.Country}</dd>
                <dt>Temp</dt><dd>{row['Max Temp']}</dd>
                <dt>Cloudiness</dt><dd>{row.Cloudiness}%</dd>
                <dt>Hotel Name</dt><dd>{row.Hotel}</dd>
                </dl>
                """
    
    hotelinfo.append(info_box)

# Display figure


In [16]:
#make plot
fighotel = gmaps.figure(map_type="HYBRID")

# Add markers
markers = gmaps.marker_layer(coordinates, info_box_content=hotelinfo )

# Add marker layer
fighotel.add_layer(markers)

# Display figure
fighotel

Figure(layout=FigureLayout(height='420px'))